In [2]:
!python -V

Python 3.9.6


In [1]:
!pip install pandas==1.2.5 pandas_datareader yfinance ipywidgets pandas_market_calendars 

In [3]:
import os
from datetime import datetime
import concurrent
import pandas_datareader.data as web
import pandas as pd
import datetime
import concurrent.futures
from concurrent.futures import wait, ALL_COMPLETED
import yfinance as yf
import urllib
import urllib.request
import time
from datetime import timedelta

from ipywidgets import interactive
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#frequency = output_variable.value
frequency = "D"

w=117*8
#start = datetime.datetime(2010, 1, 1)

#end = datetime.datetime(2013, 1, 27)
end_date = datetime.date.today()

start_date = end_date - timedelta(weeks=w)

pd.set_option('display.max_columns', None) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', None) #replace n with the number of rows you want to see completely

#cores = int(len(os.sched_getaffinity(0)))

print(end_date)
print(start_date)



etf_commodities = ['DBO','CORN', 'WEAT', 'SOYB', 'JO', 'SGG', 'BAL', 'COW', 'MOO', 'TAGS', 'KOL' ]
#Gold, Silver, Platinum, Copper, Paladium, Aluminum, Iron, Steel
etf_metals = ['IAU', 'SLV', 'PGM', 'JJC', 'PALL', 'JJU', 'IFUNX', 'SLX']
#US dollar, European Euro, Japanese yen, Pound sterling, Australian dollar, Canadian dollar, Swiss franc, Chinese Yuan Renminbi, Swedish Krona, Peso, India
#defunct: Russia: XRU, Mexico: FXM
etf_foreign_exchanges = ['UUP','FXE','FXY','FXB','FXA','FXC','FXF','CYB', 'FXS', 'INR']
#residential, Ishares all NAmerica
etf_real_estate = ['REZ', 'IYR']
#Russia, Germany, UK, Japan, China, Euro, Euro, Brazil, Latin America, Mexico, India
etf_economies = ['ERUS','EWG','EWU','EWJ','MCHI','EZU','IEUR','EWZ','ILF','EWW','INDA']
#Ishares Investment Grade, IShares core aggregate Investment grade, Short, Total, 1-5 Years, 5-10 Years, 10 Years, Gov/Credit
#defunct:

etf_spdr_indexes = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
etf_dow_components = ['MMM','AXP','AMGN','AAPL','BA','CAT','CVX','CSCO','KO','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PG','CRM','TRV','UNH','VZ','V','WMT','WBA','DIS']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

crypto = ['BTC-USD','ETH','RPL','BCH','EOS','LTC']

#,'GOLDAMGBD228NLBM',
FRED_Indicators = ['DGS2','CPALTT01USQ657N','PAYEMS','IRLTLT01USM156N','MABMM301USM189S','LFWA64TTUSM647S','MANMM101USA189S','MICH','UMCSENT','CSCICP03USM665S','DGS10','DTB3','DGS3MO','CASTHPI','GDPC1','CIVPART','POPTOTUSA647NWDB','MEHOINUSA672N','HOSMEDUSM052N','MORTGAGE30US','TTLHH','CSUSHPINSA','EMRATIO','CPIAUCSL','PSAVERT','LRUN64TTUSQ156S','USSTHPI','NYSTHPI','M2V','GFDEBTN','DFII10','GFDEGDQ188S','CUSR0000SEHA','ETOTALUSQ176N','ERENTUSQ176N','RECPROUSM156N','T5YIFR','BAMLHYH0A0HYM2TRIV','BAMLCC0A1AAATRIV','GVZCLS','DGS1','BAMLCC0A4BBBTRIV','VXVCLS','IC4WSA','WILLMICROCAPPR','WILLLRGCAPVAL','CFNAIDIFF','MZMSL','KCFSI','T5YIE','TOTALSA','USSLIND','AWHAETP','CES0500000003','TCU','WTB3MS','WGS3MO','TWEXB','DEXCHUS','DEXUSUK','CILACBQ158SBOG','CES4348400001','FEDFUNDS','TDSP','PERMIT','GFDEGDQ188S','CP','PRFI','DRSFRMACBS','DRCCLACBS','DRBLACBS','DALLCIACBEP','USROA','USROE','RSAHORUSQ156S','MEFAINUSA672N','COMREPUSQ159N','HDTGPDUSQ163N','POP','NROU','FGCCSAQ027S','TEDRATE', 'VIXCLS', 'NFCI','INDPRO','LES1252881600Q','CUUR0000SEHA','LEU0252918500Q','BAA10Y','BAMLC0A0CM','BAMLH0A3HYC','BOGMBASE','DCOILBRENTEU','DCOILWTICO','DFF','DGS1MO','DGS30','DGS5','FPCPITOTLZGUSA','ICSA','INTDSRUSM193N','M1','M1V','MPRIME','PPIACO','SPCS20RSA','STLFSI2','T10Y2Y','T10Y3M','TB3MS','TREAST','UNRATE','WPU0911']
#FRED_Indicators = ["CPALTT01USQ657N","PAYEMS","IRLTLT01USM156N","MABMM301USM189S","LFWA64TTUSM647S","MANMM101USA189S","MICH","UMCSENT","CSCICP03USM665S","DGS10","DTB3","DGS3MO","CASTHPI","GDPC1","CIVPART","POPTOTUSA647NWDB","MEHOINUSA672N","HOSMEDUSM052N","MORTGAGE30US","TTLHH","CSUSHPINSA","EMRATIO","CPIAUCSL","PSAVERT","LRUN64TTUSQ156S","USSTHPI","NYSTHPI","M2V","GFDEBTN","DFII10","GFDEGDQ188S","CUSR0000SEHA","ETOTALUSQ176N","ERENTUSQ176N","RECPROUSM156N","T5YIFR","BAMLHYH0A0HYM2TRIV","BAMLCC0A1AAATRIV","GVZCLS","DGS1","BAMLCC0A4BBBTRIV","VXVCLS","IC4WSA","WILLMICROCAPPR","WILLLRGCAPVAL","CFNAIDIFF","MZMSL","KCFSI","T5YIE","TOTALSA","USSLIND","AWHAETP","CES0500000003","TCU","WTB3MS","WGS3MO","TWEXB","DEXCHUS","DEXUSUK","CILACBQ158SBOG","CES4348400001","FEDFUNDS","TDSP","PERMIT","GFDEGDQ188S","CP","PRFI","DRSFRMACBS","DRCCLACBS","DRBLACBS","DALLCIACBEP","USROA","USROE","RSAHORUSQ156S","MEFAINUSA672N","COMREPUSQ159N","HDTGPDUSQ163N","POP","NROU","FGCCSAQ027S","TEDRATE", "VIXCLS", "NFCI","INDPRO","LES1252881600Q","CUUR0000SEHA","LEU0252918500Q","BAA10Y","BAMLC0A0CM","BAMLH0A3HYC","BOGMBASE","DCOILBRENTEU","DCOILWTICO","DFF","DGS1MO","DGS30","DGS5","FPCPITOTLZGUSA","GOLDAMGBD228NLBM","ICSA","INTDSRUSM193N","M1","M1V","MPRIME","PPIACO","SPCS20RSA","STLFSI2","T10Y2Y","T10Y3M","TB3MS","TREAST","UNRATE","WPU0911"]

Indexes = ['^SP500TR', 'QQQ', 'DIA', 'VTWO']

etf_indexes_and_Crypto_list = [Indexes, etf_commodities, etf_metals, etf_foreign_exchanges, etf_real_estate, etf_economies, etf_bonds, etf_muni_bonds, etf_treasuries, crypto, etf_spdr_indexes, etf_dow_components]

commodities = []
for sublist in etf_indexes_and_Crypto_list:
    for val in sublist:
        commodities.append(val)

#pool2 = concurrent.futures.ProcessPoolExecutor(cores)

completed = []
def dl(name):
    subset = yf.download(name, start=start_date, end=end_date, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[start_date.strftime('%Y-%m-%d'):end_date.strftime('%Y-%m-%d')]
    #sleep(4)
    if len(subset) != 0:
        completed.append(name)
        return (subset)

#futures2 = [pool2.submit(dl, args) for args in commodities]
#wait(futures2, timeout=4, return_when=ALL_COMPLETED)

futures2 = []
#pd.DataFrame()
for i in commodities:
    futures2.append(dl(i))

#print(futures2.describe())


2021-07-05
2003-07-28
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***********

In [5]:
import pandas_market_calendars as mcal

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Show available calendars
print(mcal.get_calendar_names())
early = nyse.schedule(start_date, end_date)


['ASX', 'BMF', 'CFE', 'NYSE', 'stock', 'NASDAQ', 'BATS', 'CME_Equity', 'CBOT_Equity', 'CME_Agriculture', 'CBOT_Agriculture', 'COMEX_Agriculture', 'NYMEX_Agriculture', 'CME_Rate', 'CBOT_Rate', 'CME_InterestRate', 'CBOT_InterestRate', 'CME_Bond', 'CBOT_Bond', 'EUREX', 'HKEX', 'ICE', 'ICEUS', 'NYFE', 'JPX', 'LSE', 'OSE', 'SIX', 'SSE', 'TSX', 'TSXV', 'BSE', 'TASE', 'ASEX', 'BVMF', 'CMES', 'IEPA', 'XAMS', 'XASX', 'XBKK', 'XBOG', 'XBOM', 'XBRU', 'XBUD', 'XBUE', 'XCBF', 'XCSE', 'XDUB', 'XFRA', 'XETR', 'XHEL', 'XHKG', 'XICE', 'XIDX', 'XIST', 'XJSE', 'XKAR', 'XKLS', 'XKRX', 'XLIM', 'XLIS', 'XLON', 'XMAD', 'XMEX', 'XMIL', 'XMOS', 'XNYS', 'XNZE', 'XOSL', 'XPAR', 'XPHS', 'XPRA', 'XSES', 'XSGO', 'XSHG', 'XSTO', 'XSWX', 'XTAE', 'XTAI', 'XTKS', 'XTSE', 'XWAR', 'XWBO', 'us_futures', '24/7', '24/5']


DatetimeIndex(['2003-07-28', '2003-07-29', '2003-07-30', '2003-07-31',
               '2003-08-01', '2003-08-04', '2003-08-05', '2003-08-06',
               '2003-08-07', '2003-08-08',
               ...
               '2021-06-21', '2021-06-22', '2021-06-23', '2021-06-24',
               '2021-06-25', '2021-06-28', '2021-06-29', '2021-06-30',
               '2021-07-01', '2021-07-02'],
              dtype='datetime64[ns]', length=4516, freq=None)

In [6]:

commodities_ = pd.DataFrame()

for x in range(0,len(completed)):
    values = futures2[x]
    values.index = pd.to_datetime(values.index)
    values = values.resample(frequency).mean().dropna()
    values['Symbol'] = x
    values = values.loc[~values.index.duplicated(keep='last')]
    values = values.reset_index()

    commodities_ = pd.concat([commodities_,values], axis=0)



In [7]:
import re

commodities_pvt = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
commodities_pvt.columns = completed
wd = os.getcwd()

commodities_pvt.to_csv(re.sub('code', 'data', wd)+"\commodities.csv")


In [ ]:
#commodities_pvt.describe()

In [8]:
def Fred_Data(name):
    temp = web.DataReader(str(name), 'fred', start_date, end_date)
    temp.index = pd.to_datetime(temp.index)
    temp = temp.resample(frequency).mean().dropna()
    return(temp)

#pool1 = concurrent.futures.ProcessPoolExecutor(cores)

#futures1 = [pool1.submit(Fred_Data, args) for args in FRED_Indicators]
#wait(futures1, timeout=None, return_when=ALL_COMPLETED)

FRED_set = []
FRED_completed = []
for i in FRED_Indicators:
    FRED_completed.append(i)
    FRED_set.append(Fred_Data(i))



In [9]:

FRED_pvt = pd.DataFrame()

for x in range(0,len(FRED_completed)):
    values = FRED_set[x]
    #values.index = pd.to_datetime(FRED_set["DATE"])
    values = values.resample(frequency).mean().dropna()
    #values['Symbol'] = x
    values = values.loc[~values.index.duplicated(keep='last')]
    #values = values.reset_index()

    FRED_pvt = pd.concat([FRED_pvt,values], axis=1)

#FRED_ = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
#FRED_.to_csv("/mnt/distvol/FRED_set.csv")



In [10]:

#FRED_pvt = pd.pivot_table(FRED_, index=['DATE'])
FRED_pvt.to_csv(re.sub('code', 'data', wd)+"\FRED_pvt.csv")

#print(len(FRED_.columns))
#print(len(FRED_))
#FRED_pvt.columns = FRED_completed
#FRED_pvt.describe()

In [ ]:
#FRED_pvt.set_index(FRED_pvt.index)

In [11]:
combined_set = pd.concat([FRED_pvt.set_index(FRED_pvt.index),commodities_pvt],axis=1)

In [12]:
if True:
    combined_set = combined_set.interpolate(method='linear', limit_direction='forward', axis=0)

In [16]:
combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<(len(combined_set)-1)]

Index(['CPALTT01USQ657N', 'PAYEMS', 'IRLTLT01USM156N', 'MABMM301USM189S',
       'LFWA64TTUSM647S', 'MANMM101USA189S', 'MICH', 'UMCSENT',
       'CSCICP03USM665S', 'CASTHPI',
       ...
       'SHV', 'IEI', 'BTC-USD', 'RPL', 'EOS', 'XLC', 'XLRE', 'DOW', 'CRM',
       'V'],
      dtype='object', length=142)

In [14]:
combined_set['^SP500TR'].describe()

count    6553.000000
mean     3309.496665
std      1779.947734
min      1095.040039
25%      1920.000000
50%      2421.563314
75%      4338.620117
max      9058.660156
Name: ^SP500TR, dtype: float64

In [18]:
len(drops)

142

In [15]:

drops = combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<(len(combined_set)-1)]
print(drops)
filtered = combined_set.columns.tolist()

for d in drops:
    #print(d)
    filtered.remove(d)
#filtered.remove(drops.tolist())
#combined_set[filtered].to_csv("/mnt/distvol/combined_set.csv")

#drop first/last row
all_data = combined_set[filtered]
all_data = all_data.iloc[:-1,:]
all_data = all_data.iloc[1:,:]

all_data.loc[:, (all_data != all_data.iloc[0]).any()] 

all_data.to_csv(re.sub('code', 'data', wd)+"\combined_set.csv")
#filtered
all_data.describe()

Index(['CPALTT01USQ657N', 'PAYEMS', 'IRLTLT01USM156N', 'MABMM301USM189S',
       'LFWA64TTUSM647S', 'MANMM101USA189S', 'MICH', 'UMCSENT',
       'CSCICP03USM665S', 'CASTHPI',
       ...
       'SHV', 'IEI', 'BTC-USD', 'RPL', 'EOS', 'XLC', 'XLRE', 'DOW', 'CRM',
       'V'],
      dtype='object', length=142)


,DGS2,DGS10,DTB3,DGS3MO,DFII10,T5YIFR,BAMLHYH0A0HYM2TRIV,BAMLCC0A1AAATRIV,DGS1,BAMLCC0A4BBBTRIV,WILLMICROCAPPR,WILLLRGCAPVAL,T5YIE,DEXCHUS,DEXUSUK,TEDRATE,VIXCLS,BAA10Y,BAMLC0A0CM,BAMLH0A3HYC,DCOILBRENTEU,DCOILWTICO,DFF,DGS1MO,DGS30,DGS5,M1,T10Y2Y,T10Y3M,^SP500TR,QQQ,DIA,IYR,EWG,EWU,EWJ,EZU,EWZ,ILF,EWW,LQD,SHY,IEF,TLT,ETH,BCH,LTC,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,GS,HD,HON,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WMT,WBA,DIS
count,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000
mean,1.630357,2.923349,1.203074,1.229125,0.876148,2.253299,893.924437,522.837402,1.431333,620.852948,9222.933673,51364.628251,1.841356,6.947394,1.580536,0.419207,18.887834,2.560372,1.610442,10.235217,70.903477,67.243343,1.316033,1.167639,3.599526,2.254943,3336.615326,1.292978,1.694205,3308.904046,91.874128,138.257188,51.206422,20.831823,24.818579,42.294843,30.242215,34.514051,26.602381,40.743768,85.029388,75.141306,83.341929,90.367948,18.673220,15.629086,22.667232,58.201790,32.438725,46.467687,17.536659,48.515981,41.387458,36.142230,38.252239,31.704412,100.412816,60.929433,99.105156,24.199641,118.348721,74.158175,67.256983,23.191252,28.273123,151.494035,83.925476,72.916061,103.108513,25.461947,74.039872,52.059828,83.731923,37.840576,53.196281,37.828559,61.839353,68.658589,102.212801,30.442869,60.780376,42.057614,63.432684
std,1.451914,1.140670,1.506998,1.545606,0.972315,0.366169,327.464258,120.197716,1.544442,192.069213,3474.813469,23627.815134,0.558178,0.690241,0.227235,0.418199,8.994002,0.778313,0.961931,4.828230,26.665761,23.360712,1.617540,1.516774,1.049914,1.262832,3863.546966,0.872570,1.117213,1778.651911,73.823837,72.305214,19.861505,6.083139,4.952785,9.818128,7.267728,12.984019,8.714384,12.596876,23.717682,7.391503,20.075835,31.001170,4.856237,7.793184,12.311921,39.173246,16.175476,13.491895,6.377033,28.405532,21.448073,15.064479,29.738305,14.723450,53.011316,30.378545,62.785165,29.926791,99.556930,43.058050,26.478396,11.317185,12.295209,56.269471,77.740983,52.789931,32.742514,14.185545,38.509826,33.302878,60.078599,19.001638,56.619601,33.587017,27.860748,39.326498,96.384690,14.299239,29.347534,17.670395,44.286525
min,0.090000,0.520000,-0.050000,0.000000,-1.080000,0.430000,393.960000,340.080000,0.040000,344.640000,2823.500000,17870.590000,-2.240000,6.040200,1.149200,0.080000,9.140000,1.480000,0.790000,4.140000,9.120000,-36.980000,0.040000,0.000000,0.990000,0.190000,1285.800000,-0.190000,-0.640000,1095.040039,22.676090,49.059757,13.859303,8.266589,11.260285,22.627459,14.379928,6.171552,5.784298,10.448798,49.902027,60.527298,49.728081,44.622471,4.732347,2.742188,3.513333,13.498041,12.602289,14.491638,3.999662,17.550043,11.870819,13.757485,10.777056,10.667521,29.974220,8.382245,31.408234,0.301427,21.290636,15.683257,18.180218,10.042949,11.653638,43.342461,13.207274,16.642052,46.299591,8.350197,29.727833,11.771631,13.213058,12.989122,11.554865,5.036287,26.525723,20.568348,13.676773,12.400964,29.993250,15.780844,13.424538
25%,0.450000,2.040000,0.080000,0.080000,0.221250,2.010

In [ ]:
all_data.reindex(early.index).interpolate(method='time').describe()

In [17]:
#import matplotlib.pyplot as plt
#plt.matshow(
df = combined_set.loc[combined_set.index>='2014-09-30']

r_ = df.corr()

filter = r_['BTC-USD']
                       
filter = filter[filter<1]
filter = filter.sort_values(kind="quicksort", ascending=True)
print(filter.head(10))
print(filter.tail(10))


M1V            -0.713447
NROU           -0.709515
M2V            -0.704186
DFII10         -0.612462
FXS            -0.542849
CIVPART        -0.540054
TDSP           -0.507043
MORTGAGE30US   -0.505478
WBA            -0.475707
DRSFRMACBS     -0.463185
Name: BTC-USD, dtype: float64
^SP500TR    0.839746
QQQ         0.841587
GS          0.842062
JJC         0.857388
DIS         0.865945
CAT         0.875857
VTWO        0.879186
XLB         0.880393
XLC         0.882754
MOO         0.885415
Name: BTC-USD, dtype: float64
